In [1]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
from geopy import distance
import numpy as np

In [2]:
geolocator = Nominatim(user_agent="test_tk")

In [3]:
bigdf = pd.read_csv('./results/5_years_merged_with_weather_and_311.csv')

In [4]:
POLICE_DF = pd.read_csv('./Police_Stations_20240120.csv')

In [5]:
bigdf['latitude']

0                NaN
1                NaN
2                NaN
3                NaN
4          41.818171
             ...    
1181134    41.821173
1181135    41.879292
1181136    41.920916
1181137    41.885373
1181138    41.824614
Name: latitude, Length: 1181139, dtype: float64

In [6]:
def find_closest(lat,long):
    if lat == 0 or long == 0:
        return 'No Lat/Long', 999999
    else:
        try:
            input_latlong = (long,lat)
            distance_list = []

            for index, row in POLICE_DF.iterrows():
                distance_holder = {}
                station_latlong = [row['LATITUDE'], row['LONGITUDE']]
                distance_holder['station'] = row['ADDRESS']
                distance_holder['distance'] = distance.distance(input_latlong, station_latlong).miles

                distance_list.append(distance_holder)
                distance_df = pd.DataFrame(distance_list)
                nearest_station = distance_df.iloc[distance_df['distance'].idxmin()]
                return nearest_station['station'], nearest_station['distance'] 
        except:
            return 'Error', 999999

In [16]:
longitude

latitude

41.887904127

In [20]:
closest, closestd = find_closest(bigdf.iloc[300]['longitude'], bigdf.iloc[300]['latitude'])

In [23]:
bigdf[['closest_station', 'closest_station_distance']] = bigdf[bigdf['latitude'].isnull() == False].apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_type='expand')

In [25]:
bigdf.to_csv('./results/5_years_merged_with_weather_and_311and_policedistance.csv', index=False)